In [102]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd
#from tqdm import tqdm
#from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers import Dropout, Lambda
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape,Conv2DTranspose
from tensorflow.keras.utils import load_img, img_to_array
from keras import backend as K
import tensorflow as tf
import h5py

In [103]:

def mean_iou(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.int32)
    metric = tf.keras.metrics.MeanIoU(num_classes=2)
    metric.update_state(y_true, y_pred)
    result = metric.result()
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.local_variables_initializer())
        result = sess.run(result)
    return result


In [104]:
def build_autoencoder():
    
    imh=128
    imw=128
    input_img = Input((imh, imw, 1), name='input_image')
    s = Lambda(lambda x: x / 255) (input_img)

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (s)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

    gap = tf.keras.layers.GlobalAveragePooling2D()(c5)
    radius_output = Dense(1, name='radius_output',activation='linear')(gap)
    print(radius_output) 

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)

    # outputs = Conv2D(1, (1, 1), activation='sigmoid', name='segmentation_output') (c9)
    output = Conv2D(1, (1, 1), activation='sigmoid', name='segmentation_output') (c9)
    print(input_img.shape)

    # Create model with two outputs
    model = Model(inputs=input_img, outputs=[output, radius_output])
    # model.summary()
    return model


In [105]:
# def train_autoencoder(model, x_train, radii_train, x_val, radii_val, epochs=50, batch_size=32):
#     lr = 0.0000001
#     adam_m = keras.optimizers.Adam(learning_rate=lr,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0)
#     tbCallBack = TensorBoard(histogram_freq=0,write_graph=False, write_images=False)
#     model.compile(
#         optimizer=adam_m,
#         loss={'segmentation_output': 'binary_crossentropy', 'radius_output': 'mse'},
#         metrics={'segmentation_output': [mean_iou]}
#     )
#     earlystopper = EarlyStopping(patience=100, verbose=1)
#     print("x_train shape:", x_train.shape)
#     print("radii_train shape:", radii_train.shape)
#     print("x_val shape:", x_val.shape)
#     print("radii_val shape:", radii_val.shape)

#     history = model.fit(
#         x_train,
#         {'segmentation_output': x_train, 'radius_output': radii_train},
#         validation_data=(x_val, {'segmentation_output': x_val, 'radius_output': radii_val}),
#         epochs=1500,
#         batch_size=16
#     )
#     model.summary()
def train_autoencoder(model, x_train, radii_train, x_val, radii_val, epochs=50, batch_size=32):
    lr = 0.0000001
    adam_m = keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    tbCallBack = TensorBoard(histogram_freq=0, write_graph=False, write_images=False)
    model.compile(
        optimizer=adam_m,
        loss={'segmentation_output': 'mse', 'radius_output': 'mse'}
        # metrics={'segmentation_output': [mean_iou]}
    )
    earlystopper = EarlyStopping(patience=100, verbose=1)
    print("x_train shape:", x_train.shape)
    print("radii_train shape:", radii_train.shape)
    print("x_val shape:", x_val.shape)
    print("radii_val shape:", radii_val.shape)

    history = model.fit(
        x_train,
        {'segmentation_output': x_train, 'radius_output': radii_train},
        validation_data=(x_val, {'segmentation_output': x_val, 'radius_output': radii_val}),
        epochs=1500,
        batch_size=16,
        callbacks=[earlystopper, tbCallBack]
    )
    model.summary()

    return history

    # return history


In [106]:



df = pd.read_excel('../combined_results.xlsx')

df['Image Path'] = df['Image Path'].apply(lambda x: '.' + x)
image_paths = df['Image Path'].tolist()  
radii = df['circle_radius'].values       


img_height, img_width = 128,128
images = []

for path in image_paths:
  
    img = load_img(path, target_size=(img_height, img_width), color_mode='grayscale')
    img_array = img_to_array(img) / 255.0  
    images.append(img_array)


images = np.array(images)  # Shape: (num_samples, img_height, img_width, 1)
radii = radii.reshape(-1, 1)  # Shape: (num_samples, 1)

x_train, x_val, radii_train, radii_val = train_test_split(
    images, radii, test_size=0.2, random_state=42
)


autoencoder = build_autoencoder() 
history = train_autoencoder(autoencoder, x_train, radii_train, x_val, radii_val, epochs=50, batch_size=32)


autoencoder.save('diffraction_autoencoder_doc.h5')


test_images = x_val[:10]  
reconstructed_images, predicted_radii = autoencoder.predict(test_images)


print("Predicted Radii:", predicted_radii.flatten())


<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_906>
(None, 128, 128, 1)
x_train shape: (177, 128, 128, 1)
radii_train shape: (177, 1)
x_val shape: (45, 128, 128, 1)
radii_val shape: (45, 1)
Epoch 1/1500


c:\Users\Tautvydas\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/mse_1/sub/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Tautvydas\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\Tautvydas\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\Tautvydas\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\Tautvydas\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\Tautvydas\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "c:\Users\Tautvydas\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1987, in _